In [1]:
!pip install datasets --quiet
!pip install torchvision --quiet
!pip install scikit-image --quiet
!pip install matplotlib --quiet
!pip install transformers --quiet
!pip install chromadb --quiet

In [2]:
import os
import torch
import skimage
import numpy as np
import pandas as pd
from PIL import Image
from io import BytesIO
import matplotlib.pyplot as plt
from collections import OrderedDict
from transformers import CLIPProcessor, CLIPModel, CLIPTokenizer
import requests

In [ ]:
def get_model_info(model_ID, device):
	model = CLIPModel.from_pretrained(model_ID).to(device)
	processor = CLIPProcessor.from_pretrained(model_ID)
	tokenizer = CLIPTokenizer.from_pretrained(model_ID)
	return model, processor, tokenizer

def get_single_image_embedding(my_image):
  image = processor(
      text = None,
      images = my_image,
      return_tensors="pt"
  )["pixel_values"].to(device)
  embedding = model.get_image_features(image)
  return embedding

def get_single_text_embedding(text):
  inputs = tokenizer(text, return_tensors = "pt")
  text_embeddings = model.get_text_features(**inputs)
  return text_embeddings

device = "cuda" if torch.cuda.is_available() else "cpu"
model_ID = "openai/clip-vit-base-patch32"

model, processor, tokenizer = get_model_info(model_ID, device)

# En utilisant le jeu de données sur HF

In [ ]:
from datasets import load_dataset
dataset_name = "IGNF/FLAIR_1_osm_clip"
dataset = load_dataset(dataset_name, split='train')

# Manuellement avec un mini sample

In [1]:
3

3